### Data Collection





In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
import os
import json
from requests_oauthlib import OAuth1Session
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
#from newsapi import NewsApiClient
from pathlib import Path
import tweepy as tw
%matplotlib inline
load_dotenv()

[nltk_data] Downloading package vader_lexicon to C:\Users\Straw
[nltk_data]     Hat\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
consumer_key=os.environ.get("TWITTER_API_KEY")
consumer_secret=os.environ.get("TWITTER_SECERET_API_KEY")
access_token=os.environ.get("ACCESS_TOKEN")
access_token_secret=os.environ.get("ACCESS_TOKEN_SECRET")
bearer_token =os.environ.get("TWITTER_BEARER_TOKEN")

In [4]:
import requests

client = tw.Client( bearer_token, 
                        consumer_key, 
                        consumer_secret, 
                        access_token, 
                        access_token_secret, 
                        return_type = requests.Response,
                        wait_on_rate_limit=True)

In [5]:
# Define query
query = 'from:zerohedge -is:retweet'

# get max. 10 tweets
tweets = client.search_recent_tweets(query=query, 
                                    tweet_fields=['author_id', 'created_at'],
                                     max_results=100)

In [6]:
# Save data as dictionary
tweets_dict = tweets.json() 

# Extract "data" value from dictionary
tweets_data = tweets_dict['data'] 

# Transform to pandas Dataframe
df = pd.json_normalize(tweets_data) 

In [7]:
#twitter API only limited to 7 days of feeds
df

,created_at,id,text,author_id
0,2022-03-12T14:55:14.000Z,1502659488347791366,A Visual Guide To Europe's Member States https...,18856867
1,2022-03-12T14:20:16.000Z,1502650687544508420,Rickards: The Sanctions Boomerang &amp; Putin'...,18856867
2,2022-03-12T14:00:18.000Z,1502645662026018816,RUSSIA'S RYABKOV SAYS RUSSIAN FEDERATION IS SA...,18856867
3,2022-03-12T13:59:21.000Z,1502645425836367874,UKRAINE'S ZELENSKIY SAYS WEST NOT INVOLVED ENO...,18856867
4,2022-03-12T13:45:15.000Z,1502641876263915520,"Amid Ukraine War, Russia ""Ready"" To Resume Str...",18856867
...,...,...,...,...
95,2022-03-11T16:03:08.000Z,1502314188517093379,U.S. HOUSE SPEAKER PELOSI SAYS HOUSE WILL VOTE...,18856867
96,2022-03-11T16:03:01.000Z,1502314159681294348,"*U.S. TREASURY 2-YEAR YIELD EXTENDS CLIMB, REA...",18856867
97,2022-03-11T16:00:42.000Z,1502313572994588674,HEAD OF UKRAINE'S ENERGOATOM SAYS RUSSIANS OCC...,18856867
98,2022-03-11T15:59:13.000Z,1502313202247421953,"Didi Shares Extend Collapse, Down 42% As China...",18856867


In [8]:
# Instantiate the lemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Expand the default stopwords list if necessary
sw = set(stopwords.words('english'))

In [9]:
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text
    
    regex = re.compile("[^a-zA-Z ]")
   
    # Create a tokenized list of the words
    re_clean = regex.sub('', text)
    words = word_tokenize(re_clean)
    words = word_tokenize(re_clean.lower())
    
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
   
    # Convert the words to lowercase
    words = [word for word in words if word not in sw]
    
    # Remove the stop words
   
    return lem

In [10]:
tokenizer(df.iloc[0]['text'])

['a', 'visual', 'guide', 'to', 'europe', 'member', 'state', 'httpstcoguchlao']

In [11]:
df['tokens'] = df['text'].apply(tokenizer)
df

,created_at,id,text,author_id,tokens
0,2022-03-12T14:55:14.000Z,1502659488347791366,A Visual Guide To Europe's Member States https...,18856867,"[a, visual, guide, to, europe, member, state, ..."
1,2022-03-12T14:20:16.000Z,1502650687544508420,Rickards: The Sanctions Boomerang &amp; Putin'...,18856867,"[rickards, the, sanction, boomerang, amp, puti..."
2,2022-03-12T14:00:18.000Z,1502645662026018816,RUSSIA'S RYABKOV SAYS RUSSIAN FEDERATION IS SA...,18856867,"[russia, ryabkov, say, russian, federation, is..."
3,2022-03-12T13:59:21.000Z,1502645425836367874,UKRAINE'S ZELENSKIY SAYS WEST NOT INVOLVED ENO...,18856867,"[ukraine, zelenskiy, say, west, not, involved,..."
4,2022-03-12T13:45:15.000Z,1502641876263915520,"Amid Ukraine War, Russia ""Ready"" To Resume Str...",18856867,"[amid, ukraine, war, russia, ready, to, resume..."
...,...,...,...,...,...
95,2022-03-11T16:03:08.000Z,1502314188517093379,U.S. HOUSE SPEAKER PELOSI SAYS HOUSE WILL VOTE...,18856867,"[u, house, speaker, pelosi, say, house, will, ..."
96,2022-03-11T16:03:01.000Z,1502314159681294348,"*U.S. TREASURY 2-YEAR YIELD EXTENDS CLIMB, REA...",18856867,"[u, treasury, year, yield, extends, climb, rea..."
97,2022-03-11T16:00:42.000Z,1502313572994588674,HEAD OF UKRAINE'S ENERGOATOM SAYS RUSSIANS OCC...,18856867,"[head, of, ukraine, energoatom, say, russian, ..."
98,2022-03-11T15:59:13.000Z,1502313202247421953,"Didi Shares Extend Collapse, Down 42% As China...",18856867,"[didi, share, extend, collapse, down, a, china..."


In [12]:
import spacy
from spacy import displacy

In [13]:
nlp = spacy.load('en_core_web_sm')

In [14]:
text = ' '.join(df['text'])
text

'A Visual Guide To Europe\'s Member States https://t.co/G1uCH3lAO1 Rickards: The Sanctions Boomerang &amp; Putin\'s Options https://t.co/o7DdbGid05 RUSSIA\'S RYABKOV SAYS RUSSIAN FEDERATION IS SATISFIED WITH THE DIALOGUE WITH CHINA, IT IS A RELIABLE PARTNER OF RUSSIA UKRAINE\'S ZELENSKIY SAYS WEST NOT INVOLVED ENOUGH IN PEACE NEGOTIATIONS\n\nUKRAINE\'S ZELENSKIY SAYS HE DOESN\'T SEE COMMON CONSENSUS TO ACCEPT UKRAINE INTO NATO Amid Ukraine War, Russia "Ready" To Resume Strategic Arms Reduction Talks With US https://t.co/8CSatNHScB Foreign Volunteers Who Join Fight Against Russia Can Get Citizenship: Ukraine https://t.co/3oToxaEHU2 VPN Use Surges In Ukraine And Russia https://t.co/ofg4QCMnbF CCP Seeks New Global Order At \'Expense Of All Others\': US Admiral https://t.co/YF5LWXnt2c 62-Mile-High-Club? NASA To Study Sex-In-Space, Crucial To Life On Mars https://t.co/66Gz3L0u4R Joe Biden’s Electric Car Plans Support The World’s Worst Humanitarian Abuses https://t.co/XVM13bmVQz Russia Threa

In [15]:
doc = nlp(text)

In [16]:
displacy.render(doc, style='ent')

In [17]:
from collections import Counter

# List all Entities
org_list = []

for ent in doc.ents:
    if ent.label_ == 'ORG':
        org_list.append(ent.text)


org_list = Counter(org_list).most_common(15)
df_org = pd.DataFrame(org_list, columns = ['text', 'count'])

df_org

,text,count
0,JPM,2
1,U.N.,2
2,The Sanctions Boomerang &,1
3,Options https://t.co/o7DdbGid05 RUSSIA'S,1
4,NASA,1
5,Crucial To Life On Mars https://t.co/66Gz3L0u4...,1
6,Updates Search Engine,1
7,Collapse,1
8,Gun Rights Group,1
9,Become World's,1


In [18]:
df.to_csv('tweets1.csv')

In [ ]:
#second tweet collection 
# Define query
query2 = 'from:zerohedge -is:retweet'

# get max. 10 tweets
tweets2 = client.search_recent_tweets(query2=query2, 
                                    tweet_fields=['author_id', 'created_at'],
                                     max_results=100)

In [ ]:
# Save data as dictionary
tweets_dict2 = tweets2.json() 

# Extract "data" value from dictionary
tweets_data2 = tweets_dict2['data'] 

# Transform to pandas Dataframe
df = pd.json_normalize(tweets_data) 